### Tensor shape 변경
- reshape(), view() : 원소 개수가 유지됨, 기존의 텐서를 공유함

### (1) 모듈 로딩 

In [1]:
from torch import *
import torch
import intel_extension_for_pytorch as ipex

In [2]:
t1 = tensor(
    [
        [1,2,3],
        [4,5,6]
    ]
)
print(t1.shape, t1.ndim)

torch.Size([2, 3]) 2


In [3]:
t1.reshape((3,2))

tensor([[1, 2],
        [3, 4],
        [5, 6]])

In [4]:
t1.view((3,2))

tensor([[1, 2],
        [3, 4],
        [5, 6]])

In [5]:
t1.reshape((6,1))

tensor([[1],
        [2],
        [3],
        [4],
        [5],
        [6]])

In [6]:
# [2,3] ==> [6,1] 형태 변경 : 원소 동일 6개
# -1 : 원소수를 알아서 특정 차원을 조절
t1.view((-1,1))

tensor([[1],
        [2],
        [3],
        [4],
        [5],
        [6]])

In [7]:
# 원소 수 유지되어야 함!!
# RuntimeError: shape '[-1, 7]' is invalid for input of size 6
t1.reshape((3,-1))

tensor([[1, 2],
        [3, 4],
        [5, 6]])

- 전치 : 열 <-> 행 전환

In [8]:
t1.shape

torch.Size([2, 3])

In [9]:
t1.T, t1

(tensor([[1, 4],
         [2, 5],
         [3, 6]]),
 tensor([[1, 2, 3],
         [4, 5, 6]]))

In [10]:
t1.T.is_contiguous(), t1.is_contiguous(), t1.reshape((-1,6)).is_contiguous()

(False, True, True)

In [11]:
t1.T.reshape((-1,6)), t1.T.reshape((-1,6)).is_contiguous()

(tensor([[1, 4, 2, 5, 3, 6]]), True)

### 텐서 데이터의 메모리 저장 정보 (metadata)
- 현재 저장 형태, 검색 방향 정보, 시작 정보

In [12]:
t1.shape, t1.ndim

(torch.Size([2, 3]), 2)

In [13]:
t1

tensor([[1, 2, 3],
        [4, 5, 6]])

In [14]:
result = t1.storage(), t1.storage_offset(), t1.stride()
method = ['storage', 'storage_offset', 'stride', 'is_contiguous', 'data_ptr']

/tmp/ipykernel_175716/1609616123.py:1: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  result = t1.storage(), t1.storage_offset(), t1.stride()


In [15]:
for x in method:
    print(f"{x} : {getattr(t1, x)()}")

storage :  1
 2
 3
 4
 5
 6
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 6]
storage_offset : 0
stride : (3, 1)
is_contiguous : True
data_ptr : 132031168


In [16]:
t1.view(-1,2)

tensor([[1, 2],
        [3, 4],
        [5, 6]])

In [17]:
for x in method:
    print(f"{x} : {getattr(t1.view(-1,2), x)()}")

storage :  1
 2
 3
 4
 5
 6
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 6]
storage_offset : 0
stride : (2, 1)
is_contiguous : True
data_ptr : 132031168


In [18]:
for x in method:
    print(  f"{x} : {getattr(t1.T.T.T.T, x)()}")

storage :  1
 2
 3
 4
 5
 6
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 6]
storage_offset : 0
stride : (3, 1)
is_contiguous : True
data_ptr : 132031168


### 차원 제거 / 추가
- tensor.squeeze() : 텐서에서 차원이 1인 것 제거
- tensor.unsqueeze(dim) : 텐서에서 차원 1인 것 추가

In [19]:
# 데이터 생성

t1 = tensor([
    [1,2],
    [3,4],
])

t2 = tensor(
    [
        [1,2,3,4]
    ]
)
t3 = tensor(
    [
        [
            [1,2,3,4],
        ]
    ]
)


In [20]:
print(f"t1 => {t1.shape}, {t1.ndim}D")
print(f"t2 => {t2.shape}, {t2.ndim}D")


t1 => torch.Size([2, 2]), 2D
t2 => torch.Size([1, 4]), 2D


In [21]:
# 차원 축소
t11 = t1.squeeze()
t22 = t2.squeeze()

print(f"t11 => {t11.shape}, {t11.ndim}D")
print(f"t22 => {t22.shape}, {t22.ndim}D")

t11 => torch.Size([2, 2]), 2D
t22 => torch.Size([4]), 1D


In [22]:
t33 = t3.squeeze()

In [23]:
print(f"t1 차원축소 => {t11.shape}, {t11.ndim}D, {t11.data_ptr()}")
print(f"t2 차원축소 => {t22.shape}, {t22.ndim}D, {t22.data_ptr()}")
print(f"t3 차원축소 => {t33.shape}, {t33.ndim}D, {t33.data_ptr()}")
print()


t1 차원축소 => torch.Size([2, 2]), 2D, 132107264
t2 차원축소 => torch.Size([4]), 1D, 132107392
t3 차원축소 => torch.Size([4]), 1D, 132076928



In [24]:
## 원소/요소 수 변경 없이 1차원 증가 시키키 => torch.unsqueeze(dim)
t1.shape

torch.Size([2, 2])

In [25]:
t11 = t1.unsqueeze(dim = 0)
print(f"1차원 추가 => {t11.shape}, {t11.ndim}D, {t11.data_ptr()}, {t11.stride()}")

1차원 추가 => torch.Size([1, 2, 2]), 3D, 132107264, (4, 2, 1)


In [26]:
t22 = t1.unsqueeze(dim = -1)
print(f"1차원 추가 => {t22.shape}, {t22.ndim}D, {t22.data_ptr()}, {t22.stride()}")

1차원 추가 => torch.Size([2, 2, 1]), 3D, 132107264, (2, 1, 1)


In [27]:
lt = LongTensor([1,2,3,4])
print(lt.dtype, lt)


torch.int64 tensor([1, 2, 3, 4])


In [28]:
lt.type(dtype=torch.float32)

tensor([1., 2., 3., 4.])